In [ ]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [ ]:
import time # 새로운 페이지로 셀레니음 웹드라이버가 넘어갈 때마다 잠시 멈춰주는 역할을 위해 불러옴. 
# 멈춰주지 않으면 페이지가 다 로딩되기도 전에 다음 코드를 실행해버리는 불상사가 생김. 그걸 막아준다.
from selenium import webdriver # 웹드라이버. 코랩이라 우리가 직접 볼 순 없지만 얘가 돌아다니면서 일을 해줌
from selenium.webdriver.common.keys import Keys # 이 파일에선 아직 딱히 쓰지 않은듯
from selenium.webdriver.common.by import By # element를 찾을 때 쓰이는 parameter를 불러오는 역할
import pandas as pd 
from bs4 import BeautifulSoup # html 요소에 접근해서 정보를 빼내올 때 씀
from tqdm import tqdm
import pickle
import numpy as np
from urllib.parse import quote # 주소창에 한글을 입력할 때 변환하는 메서드

In [ ]:
import warnings # 사실 잘 모르지만 선생님이 하셨던거 암튼 가져옴
warnings.filterwarnings('ignore')

In [ ]:
#크롬 드라이버의 각종 옵션들 생성 및 적용.
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.87 Whale/3.16.138.22 Safari/537.36') # 나는 기계가 아니고 사람이에요
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100'

In [ ]:
driver.get(url)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
buttons = soup.select('.input_check')
buttons[50]['value']

'35541726'

In [ ]:
song_url_base = 'https://www.melon.com/song/detail.htm?songId='
song_url = song_url_base + buttons[1]['value']
driver.get(song_url)

In [ ]:
text = driver.find_element(By.CSS_SELECTOR, '#d_video_summary').text.replace('\n\n','\n')
text

'착한 아이처럼\n말만 잘 들으라 해서\n시키는 대로 했는데\n자꾸 지겨워 해\n내가 봐도\n나는 정말 쉬웠어\n난 울다가도\n사랑 주면 웃었어\n늘 나를 쉽게 다루는 건\n막대사탕 같은 사랑\n향기 가득 배어 있는 단물이\n꼭 보기 좋게 물 들은 혀 끝이\n나 혼자 가진 매력인 듯\n착각 속에 빠졌지만\n착한 아이처럼\n말만 잘 들으라 해서\n시키는 대로 했는데\n자꾸 지겨워 해\n아무리 떼를 써도\n차라리 토라져봐도\n남자가 주는 이별에\n항상 울기만 해\n호기심에 다가오던 사랑이\n내게 싫증 내고서 떠나도\n아이같이 금방 잊고\n다시 사랑받길 원해\n착한 아이처럼\n말만 잘 들으라 해서\n시키는 대로 했는데\n자꾸 지겨워 해\n아무리 떼를 써도\n차라리 토라져봐도\n남자가 주는 이별에\n항상 울기만 해\n(남자들은 다) 난 믿지 않아\n(언제나 아픈 사랑도) 더 하지 않아\n(매일 밤 혼잣말로만) 날 위로해도\n난 사랑 없이 못 사나 봐\n첫 사랑에 속고\n또 다른 시작을 해도\n어느새 손을 놓은 채\n모두 떠나가네\n한숨이 늘어가고\n눈물이 나를 적셔도\n반복된 사랑놀이에\n울고 웃네'

In [ ]:
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100'

In [ ]:
driver.get(url)

In [ ]:
driver.find_element(By.XPATH, //*[@id="conts"]/div[2]/ul/li[2]/a/span).click()

In [ ]:
song_url_base = 'https://www.melon.com/song/detail.htm?songId='
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100'
sentences = [[] for _ in range(7)]
for i in range(7):
    driver.get(url)
    time.sleep(1)
    if i != 1:
        driver.find_element(By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{i+1}]/a/span').click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    buttons = soup.select('.input_check')
    for bt in tqdm(buttons):
        try:
          song_url = song_url_base+bt['value']
          driver.get(song_url)
          time.sleep(1)
          text = driver.find_element(By.CSS_SELECTOR, '#d_video_summary').text.replace('\n\n','\n')
          sentences[i].append(text.split('\n'))
        except:
          continue

In [ ]:
song_url_base = 'https://www.melon.com/song/detail.htm?songId='
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100'
sentences = [[] for _ in range(7)]
for i in range(7):
  driver.get(url)
  time.sleep(1)
  if i != 1:
    driver.find_element(By.XPATH, f'//*[@id="conts"]/div[2]/ul/li[{i+1}]/a/span').click()
  for j in range(10):
    if j != 0:
      time.sleep(1)
      driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{j}]').click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    buttons = soup.select('.input_check')
    for bt in tqdm(buttons):
      try:
        song_url = song_url_base+bt['value']
        driver.get(song_url)
        time.sleep(1)
        text = driver.find_element(By.CSS_SELECTOR, '#d_video_summary').text.replace('\n\n','\n')
        sentences[i].append(text.split('\n'))
      except:
        continue

In [ ]:
//*[@id="pageObjNavgation"]/div/span/a[1]
//*[@id="pageObjNavgation"]/div/span/a[2]

In [ ]:
#pageObjNavgation > div > span > a:nth-child(3)